In [1]:
from model.ffnn2 import FFNN2

from view.gui import GUI, GraphModel
from PyQt6.QtWidgets import QApplication
import sys
import numpy as np
import os
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score, accuracy_score
import time

MODEL_FILENAME = "ffnn_model.pkl"

In [2]:
train_samples = 5000

# Inisialisasi model
model = FFNN2(
    jumlah_neuron=[784, 128, 64, 64, 10],
    fungsi_aktivasi=["ReLU", "ReLU", "ReLU", "Softmax"],
    fungsi_loss="CategoricalCrossEntropy",
    inisialisasi_bobot="xavier-normal",
    seed=123123,
    lower_bound=-1,
    upper_bound=1,
    mean=0,
    std=1,
    verbose=1,
)

# Cek apakah model sudah ada
if os.path.exists(MODEL_FILENAME):
    model = FFNN2.load_model_pickle(model, MODEL_FILENAME) # Load model dari file
    train_needed = False
else:
    train_needed = True

In [ ]:
# Train or load

# Load MNIST dataset
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
X = X / 255.0

# Split dataset into training, validation, and test sets
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=10000, random_state=123123
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, train_size=5000, test_size=1000, random_state=42
)

# One-Hot Encoding
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))
y_val = encoder.transform(y_val.reshape(-1, 1))

if train_needed:
    # Catat waktu mulai training
    start_time = time.time()

    # Training model
    model.fit(X=X_train, y=y_train, batch=100, lr=0.1, epochs=500, X_val=X_val, y_val=y_val)

    # Catat waktu selesai training
    end_time = time.time()
    print(f"Training time: {end_time - start_time:.2f} seconds")

    # Simpan model setelah training
    model.save_model_pickle(MODEL_FILENAME)

Training model...


ValueError: operands could not be broadcast together with shapes (1000,) (1000,10) 

In [ ]:
# Prediksi
pred = model.predict(X_test)

# Konversi y_test dari one-hot encoding ke label asli
y_test_labels = np.argmax(y_test, axis=1)

# Hitung F1-score
model_f1 = f1_score(y_test_labels, pred, average="macro")
print(f"Model F1-Score: {model_f1:.4f}")

# Hitung Accuracy
model_accuracy = accuracy_score(y_test_labels, pred)
print(f"Model Accuracy: {model_accuracy:.4f}")

Model F1-Score: 0.9190
Model Accuracy: 0.9198


In [ ]:
# Loop GUI dengan perintah ini
%gui qt

In [ ]:
# Visualize model

graph_model = GraphModel(
    model.jumlah_neuron, model.bobot, model.gradients, model.training_loss_results, validation_loss=model.validation_loss_results
)
layer_distribution_input = [0,1,2,3]
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
gui = GUI(graph_model, layer_distribution_input)
gui.show()

TEST:   [np.float64(2.181986870635541), np.float64(1.995082623708847), np.float64(1.7238753457173137), np.float64(1.3836450760890942), np.float64(1.069528651457994), np.float64(0.8480792390331487), np.float64(0.7138145446313442), np.float64(0.6181304283906244), np.float64(0.5595624871270187), np.float64(0.5143051300769105), np.float64(0.48350279305362326), np.float64(0.4569430940639866), np.float64(0.43519803092175935), np.float64(0.41555428607325046), np.float64(0.4000752163707716), np.float64(0.383064750108014), np.float64(0.37270432687467436), np.float64(0.35933662117814774), np.float64(0.35620765150159006), np.float64(0.34082674615659997), np.float64(0.3318177527702578), np.float64(0.3222909400001285), np.float64(0.3198974129549515), np.float64(0.3102560929778581), np.float64(0.30066181443137574), np.float64(0.3020574359576777), np.float64(0.2876009569112043), np.float64(0.2845728868901962), np.float64(0.2777027952995619), np.float64(0.27193815209378047), np.float64(0.2652426373386